# MrPUP-8: for v2, look at the difference of claim counts between "O" NPI and  "P" NPI

QA step description: for v2 look at the difference of claim counts between "O" NPI and  "P" NPI

Starting Author: Amy Jin (amy@careset.com)

Date: July 16th, 2018

https://docs.google.com/spreadsheets/d/1IYg01IpssJaWHo6KxO4_dSDgXtYNFy41S5cIHFLvlGQ/edit#gid=604789549

## Connection to Parenthood Server

In [2]:
# Packages import
import os
import sys
import numpy as np
import pandas as pd
from collections import Counter
import operator
import mysql.connector
import sshtunnel
import pureyaml

# Handle path
project_dir = !pwd  # dir of current script/notebook file
config_file = open(project_dir[0] + "/db.yaml");
config = pureyaml.load(config_file.read());

# Argument dictionary for sshtunnel
ssh_config = {
    'ssh_address_or_host': ('parenthood.set.care', 22),
    'ssh_username':        config['ssh_username'],
    'ssh_password':        config['ssh_password'],
    'remote_bind_address': ('127.0.0.1', 3306),
    'local_bind_address':  ('0.0.0.0', 3333),
}

# Argument dictionary for mysql.connector
mysql_config = {
    'user':     config['mysql_user'],
    'password': config['mysql_passwd'],
    'host':     config['mysql_host'],
    'database': 'patch',
    'port':     3333,
}

# Connect to Parenthood server
with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
    print('SSH tunneling successful on port: {}'.format(tunnel.local_bind_port))
    connection = mysql.connector.connect(**mysql_config)
    cur = connection.cursor()
    print('MySQL server connected successfully!')

SSH tunneling successful on port: 3333
MySQL server connected successfully!


## Test Function

In [3]:
# --------------------------------------- Inputs: ---------------------------------------
# 1) db_onpi:                database name with ONPI
# 2）table_onpi:             table name with ONPI
# 3) col_onpi:               patient count of ONPI

# 4) db_pnpi:                database name with PNPI
# 5) table_pnpi:             table name with ONPI
# 6）col_pnpi：               patient count of ONPI
# --------------------------------------- Outputs: --------------------------------------
# 1) The test will print out:
#       - the total claim count of ONPI in MrPUP V2 and 
#       - the total claim count of PNPI in MrPUP V2


def mrpup_8(db_onpi, table_onpi, col_onpi, db_pnpi, table_pnpi, col_pnpi):
    #table1 = str(db_name) + '.' + str(table_name)
    with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
        connection = mysql.connector.connect(**mysql_config)
        cur = connection.cursor()
            

        # MySQL to get total claim count of onpi 
        query = ('''
                SELECT sum({col1})
                FROM {db1}.{t1} AS V1;
        ''').format(db1 = db_onpi, t1 = table_onpi, col1 = col_onpi)
        cur.execute(query)

        print ("The total {} in {}.{} is: ".format(col_onpi, db_onpi, table_onpi) + '\n')
        for row in cur.fetchall():
            for i in range(0,len(row)):
                print (str(row[i]))
            print ('\n')
            
        # MySQL to get total claim count of pnpi  
        query = ('''
                SELECT sum({col1})
                FROM {db1}.{t1} AS V1;
        ''').format(db1 = db_pnpi, t1 = table_pnpi, col1 = col_pnpi)
        cur.execute(query)

        print ("The total {} in {}.{} is: ".format(col_pnpi, db_pnpi, table_pnpi) + '\n')
        for row in cur.fetchall():
            for i in range(0,len(row)):
                print (str(row[i]))
            print ('\n')
            
        cur.close()
        connection.close()

## Test Example

In [4]:
mrpup_8('_amy', 'test_mrpup_good', 'cnt_clm_id', \
       '_amy', 'test_mrpup_bad', 'cnt_clm_id')

The total cnt_clm_id in _amy.test_mrpup_good is: 

5845


The total cnt_clm_id in _amy.test_mrpup_bad is: 

5845




## Internal Data Test Example

In [5]:
mrpup_8('mrpup_carr', 'mrpup_two_npi_rfr_prf_RQ17', 'cnt_clm_id', \
        'mrpup_carr', 'mrpup_two_npi_rfr_blg_RQ17', 'cnt_clm_id')

The number of claim count in ONPI V2 is: 

740249271


The number of claim count in PNPI V2 is: 

841441918


